In [ ]:
!pip install torch torchvision torchinfo

In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, models,transforms as T
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader

In [2]:
net =  models.resnet18()        

In [4]:
summary(net, input_size=(1,3,28,28))

if torch.cuda.is_available():
    net.cuda()

In [5]:
transform = T.Compose([T.Grayscale(num_output_channels=3), T.ToTensor()])

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 100

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        X = X.cuda()
        y = y.cuda()
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.cuda()
            y = y.cuda()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
train_loop(train_dataloader,net,loss_fn,optimizer)
test_loop(test_dataloader,net,loss_fn)

loss: 6.842259  [    0/60000]
loss: 1.818234  [ 6400/60000]
loss: 0.526251  [12800/60000]
loss: 0.507324  [19200/60000]
loss: 0.229032  [25600/60000]
loss: 0.201113  [32000/60000]
loss: 0.249871  [38400/60000]
loss: 0.266945  [44800/60000]
loss: 0.264392  [51200/60000]
loss: 0.152951  [57600/60000]
Test Error: 
 Accuracy: 95.6%, Avg loss: 0.002577 

